In [1]:
import folium
from folium.plugins import MarkerCluster

In [2]:
import numpy as np

size = 100
lons = np.random.randint(-180, 180, size=size)
lats = np.random.randint(-90, 90, size=size)
dummy = np.random.randint(0, 20, size=size)

locations = list(zip(lats, lons))
popups = ["lon:{}<br>lat:{}".format(lon, lat) for (lat, lon) in locations]

Adding all icons in a single call

In [3]:
icon_create_function = """\
function(cluster) {
    return L.divIcon({
    html: '<b>' + cluster.getChildCount() + '</b>',
    className: 'marker-cluster marker-cluster-large',
    iconSize: new L.Point(20, 20)
    });
}"""

Explicit loop allow for customization in the loop.

In [4]:
%%time

m = folium.Map(
    location=[np.mean(lats), np.mean(lons)],
    tiles='Cartodb Positron',
    zoom_start=1
)

marker_cluster = MarkerCluster(
    name='1000 clustered icons',
    overlay=True,
    control=False,
    icon_create_function=None
)

for k in range(size):
    location = lats[k], lons[k]
    marker = folium.Marker(location=location)
    popup = 'lon:{}<br>lat:{}'.format(location[1], location[0])
    folium.Popup(popup).add_to(marker)
    marker_cluster.add_child(marker)

marker_cluster.add_to(m)

folium.LayerControl().add_to(m);

Wall time: 27.9 ms


In [5]:
m

In [7]:
marker_data =[]
for lng, lat, label in zip(lons, lats, dummy):
    marker_data.append({'location':[lat,lng],'population':round(label)})

marker_data=tuple(marker_data)
marker_data

({'location': [55, 36], 'population': 14},
 {'location': [21, -165], 'population': 6},
 {'location': [-66, -75], 'population': 5},
 {'location': [40, 111], 'population': 4},
 {'location': [20, -142], 'population': 18},
 {'location': [-35, -31], 'population': 10},
 {'location': [-70, 149], 'population': 4},
 {'location': [64, 149], 'population': 18},
 {'location': [-23, -62], 'population': 8},
 {'location': [-77, -48], 'population': 18},
 {'location': [70, 75], 'population': 7},
 {'location': [-2, 100], 'population': 2},
 {'location': [-20, -96], 'population': 3},
 {'location': [4, 124], 'population': 13},
 {'location': [-62, -2], 'population': 7},
 {'location': [15, -82], 'population': 4},
 {'location': [30, 135], 'population': 13},
 {'location': [23, -78], 'population': 19},
 {'location': [-4, 113], 'population': 4},
 {'location': [61, -96], 'population': 7},
 {'location': [84, 158], 'population': 14},
 {'location': [6, 9], 'population': 4},
 {'location': [23, 19], 'population': 0},
 

In [8]:
import json
import folium
from folium import Marker
from folium.plugins import MarkerCluster
from jinja2 import Template


class MarkerWithProps(Marker):
    _template = Template(u"""
        {% macro script(this, kwargs) %}
        var {{this.get_name()}} = L.marker(
            [{{this.location[0]}}, {{this.location[1]}}],
            {
                icon: new L.Icon.Default(),
                {%- if this.draggable %}
                draggable: true,
                autoPan: true,
                {%- endif %}
                {%- if this.props %}
                props : {{ this.props }} 
                {%- endif %}
                }
            )
            .addTo({{this._parent.get_name()}});
        {% endmacro %}
        """)
    def __init__(self, location, popup=None, tooltip=None, icon=None, draggable=False, props = None ):
        super(MarkerWithProps, self).__init__(location=location,popup=popup,tooltip=tooltip,icon=icon,draggable=draggable)
        self.props = json.loads(json.dumps(props)) 
        
icon_create_function = '''
    function(cluster) {
        var markers = cluster.getAllChildMarkers();
        var sum = 0;
        for (var i = 0; i < markers.length; i++) {
            sum += markers[i].options.props.population;
        }
        var avg = Math.round(sum/cluster.getChildCount());
        
        var c = ' marker-cluster-';

        if (avg < 6) {
            c += 'small';
        } else if (avg < 12) {
            c += 'medium';
        } else {
            c += 'large';
        }

        return L.divIcon({
             html: '<div><span>' + avg + '</span></div>',
             className: 'marker-cluster marker-cluster' + c,
             iconSize: new L.Point(40, 40)
        });
    }
'''


marker_cluster = MarkerCluster(icon_create_function=icon_create_function)

for marker_item in marker_data:
    marker = MarkerWithProps(
        location=marker_item['location'],
        props = { 'population': marker_item['population']},
        icon=folium.Icon(color='red', icon='building-sign',prefix = 'glyphicon')
    )
    marker.add_to(marker_cluster)

In [9]:
m = folium.Map(
    location=[np.mean(lats), np.mean(lons)],
    tiles='Cartodb Positron',
    zoom_start=1
)
marker_cluster.add_to(m) 

In [10]:
m